In [1]:
import os
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import random
import difflib
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import *
from sklearn.utils import shuffle

In [2]:
epochs = 10
batch_size = 96

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
'''
the current dataset and epochs are not representive of what was used for the report but more so what was needed 
to run the code in spyder to convert from jupyter notebook to py.file. Accuracy will be lower on triplet loss CNN
as epochs and dataset were reduced. Please refer to the the jupytern notebooks attached in the submission to see the 
actual parameters used.
'''

def my_team():
    '''
    Return the list of the team members of this assignment submission as a list
    of triplet of the form (student_number, first_name, last_name)
    
    '''
    return [(9993304, "Brendan", "Wallace-Nash"),(9300449, "Min-Pu", "Tsai"), (10661450, "Bingqing", "Qian")]
    raise NotImplementedError()

def triplet_loss_test():
    '''
    Read back the arrays x_train, y_train, x_test and y_test
    from the npz file named 'mnist_dataset.npz'.
    Then, print the shape and dtype of these numpy arrays.
    
    '''
    def triplet_loss(y_true, y_pred, margin = 0.4):
        """
        Implementation of the triplet loss function
        Arguments:
        y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
        y_pred -- python list containing three objects:
                anchor -- the encodings for the anchor data
                positive -- the encodings for the positive data (similar to anchor)
                negative -- the encodings for the negative data (different from anchor)
        Returns:
        loss -- real number, value of the loss
        """
    
        anchor = y_pred[0]
        positive = y_pred[1]
        negative = y_pred[2]
    
        # distance between the anchor and the positive
        pos_dist = tf.reduce_sum(tf.square(anchor-positive),axis=1)
    
        # distance between the anchor and the negative
        neg_dist = tf.reduce_sum(tf.square(anchor-negative),axis=1)
    
        # compute loss
        basic_loss = pos_dist-neg_dist+margin
        loss = tf.maximum(basic_loss,0.0)
        loss = tf.reduce_mean(loss)     
        return loss
    
    #Test implementation of triplet loss function 
    num_data = 10
    feat_dim = 6
    margin = 0.2
    
    embeddings = [np.random.rand(num_data, feat_dim).astype(np.float32),
                  np.random.rand(num_data, feat_dim).astype(np.float32),
                  np.random.rand(num_data, feat_dim).astype(np.float32)]
    labels = np.random.randint(0, 1, size=(num_data)).astype(np.float32)
      
    #Compute loss with numpy
    loss_np = 0.
    anchor = embeddings[0]
    positive = embeddings[1]
    negative = embeddings[2]
    for i in range(num_data):
        pos_dist = np.sum(np.square(anchor[i] - positive[i]))
        neg_dist = np.sum(np.square(anchor[i] - negative[i]))
        loss_np += max(pos_dist-neg_dist+margin, 0.)
    loss_np /= num_data
    print('Triplet loss computed with numpy', loss_np)
    
    # Compute the loss in TF
    tf.compat.v1.disable_eager_execution()
    loss_tf = triplet_loss(labels, embeddings, margin)
    with tf.compat.v1.Session() as sess:
        loss_tf_val = sess.run(loss_tf)
        print('Triplet loss computed with tensorflow', loss_tf_val)
    assert np.allclose(loss_np, loss_tf_val)



def data_prepare():
    '''
    Prepare the dataset for training and testing from tensorflow dataset.
    
    Arguments:
        (None)
        
    Returns:
        arrayImage -- numpy array images of alphabets
        arrayAlph -- names of alphabets 
        x_test --  numpy array images of alphabets in test data
        y_test -- names of alphabets in test data
        x_train -- images of alphabets in train data. empty variable at the moment.
        y_train -- names of alphabets in train data. empty variable at the moment.
    '''
    #Load omniglot dataset from tensorflow dataset, divide it into train data and test data by 80%.
    ds, ds_info = tfds.load('omniglot', split=['train[:80%]','test'],with_info=True, as_supervised = False)
    train = tfds.as_dataframe(ds[0])
    test = tfds.as_dataframe(ds[1])
    
    #we set this valus just to run quicker.
    train = train[:16000]
    test = test[:4000]
    
    #Create sets from train data by the attributes, name and image of alphbet in dataset;
    arrayAlph = train["alphabet"].to_list()
    arrayImage = train["image"].to_list()
    #do the same thing to test data.
    y_test = test["alphabet"].to_list()
    x_test = test["image"].to_list()
    
    #Empty at the moment
    x_train = [] 
    y_train = []
    
    #Convert images to numpy array
    x_test = np.array(x_test)
    arrayImage = np.array(arrayImage)
    
    return arrayImage, arrayAlph, x_test, y_test, x_train, y_train


def global_variable1(arrayImage, arrayAlph):  
    '''
    Training data. Prepare the variables will be used next.
    
    Arguments:
        arrayImage, arrayAlph
        
    Returns:
        label -- training label list. 'Label' here means Positive/Negtive 
                 results of corresponding triplet. Recording as 1/0               
        tripletLitst -- training ist of triplet. A triplet contains Anchor image, 
                        Positive image, and Negtive image
    '''
    label = []
    tripletLitst = []
    imageL1 = arrayImage
    alphL1 = arrayAlph
    imageL2 = arrayImage.copy()
    alphL2 = arrayAlph.copy()
    imageL3 = arrayImage.copy()
    alphL3 = arrayAlph.copy()
    
    shuffleList1 = list(zip(imageL2, alphL2))
    random.shuffle(shuffleList1)
    imageL2, alphL2 = zip(*shuffleList1)
    
    shuffleList2 = list(zip(imageL3, alphL3))
    random.shuffle(shuffleList2)
    imageL3, alphL3 = zip(*shuffleList2)
    
    for i in range(len(imageL1)):
        a = []
        p = []
        if alphL1[i] == alphL2[i]:
            a.append(imageL1[i])
            p.append(imageL2[i])
            for x in range(len(imageL1)):
                n = []
                if alphL1[i] != alphL3[x]:
                    n.append(imageL3[x])
                    label.append(0)
                    tripletLitst.append([a, p, n])
    
    tripletLitst = tripletLitst[:20000]
    label = label[:20000]
    
    
    
    label = np.array(label)
    
    tripletLitst = np.array(tripletLitst)
    
    return label, tripletLitst
    
    
def global_variable2(arrayImage, arrayAlph):  
    '''
    Testing data. Prepare the variables will be used next.
    
    Arguments:
        arrayImage, arrayAlph
        
    Returns:
        testlabel -- testing label list. 'Label' here means Positive/Negtive 
                     results of corresponding triplet. Recording as 1/0 
        testList -- testing ist of triplet. A triplet contains Anchor image, 
                    Positive image, and Negtive image
    '''
    testlabel = []
    testList = []
    imageL1 = arrayImage
    alphL1 = arrayAlph
    imageL2 = arrayImage.copy()
    alphL2 = arrayAlph.copy()
    imageL3 = arrayImage.copy()
    alphL3 = arrayAlph.copy()
    
    shuffleList1 = list(zip(imageL2, alphL2))
    random.shuffle(shuffleList1)
    imageL2, alphL2 = zip(*shuffleList1)
    
    shuffleList2 = list(zip(imageL3, alphL3))
    random.shuffle(shuffleList2)
    imageL3, alphL3 = zip(*shuffleList2)
    
    for i in range(len(imageL1)):
        a = []
        p = []
        if alphL1[i] == alphL2[i]:
            a.append(imageL1[i])
            p.append(imageL2[i])
            for x in range(len(imageL1)):
                n = []
                if alphL1[i] != alphL3[x]:
                    n.append(imageL3[x])
                    testlabel.append(0)
                    testList.append([a, p, n])
                    
                elif alphL1[i] == alphL3[x]:
                    n.append(imageL3[x])
                    testlabel.append(1)
                    testList.append([a, p, n])

    valuesTest, countsTest = np.unique(testlabel, return_counts=True)

    
    testList = testList[:4000]
    testlabel = testlabel[:4000]
    testList = np.array(testList)
    testlabel = np.array(testlabel)
    
    return testlabel, testList
 
   

def dataProcess(arrayImage, arrayAlph, x_test, y_test, tripletLitst):    
    '''
    Processing data to numpy array. Including training data nad testing data
    
    Argument:
        arrayImage -- numpy array images of alphabets
        arrayAlph -- names of alphabets 
        x_test --  numpy array images of alphabets in test data
        y_test -- names of alphabets in test data
        x_train -- images of alphabets in train data
        y_train -- names of alphabets in train data
        tripletLitst -- training ist of triplet. A triplet contains Anchor image, 
                        Positive image, and Negtive image
    Return:
        same variables but convert to numpy array, reshape and/or reduce to 
        resonable sample quantity
    '''
    lenX = round(len(tripletLitst)*0.20)
    lenY = round(len(label)*0.20)
    x_train = tripletLitst[lenX:]
    x_test = tripletLitst[lenX:]
    y_train = tripletLitst[:lenX]
    y_test = tripletLitst[:lenX]
    x_train = np.array(x_train)
    x_test = np.array(x_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    
    x_train = x_train.reshape(len(x_train), 3, 105, 105, 3)
    y_train = y_train.reshape(len(y_train), 3, 105, 105, 3)
    
    arrayAlph = np.array(arrayAlph)
    arrayImage = np.array(arrayImage)
    y_test = y_test[:16000]    
    x_test = x_test[:16000]
    
    return arrayImage, arrayAlph, x_test, y_test, x_train, y_train

def triplet_loss(y_true, y_pred):
    '''
    Triple loss function
    
    Argument:
        y_true -- exact valuse of y
        y_pred -- prediction values of y
    Return:
        K.mean
    '''
    anchor_out = y_pred[:, 0:100]
    positive_out = y_pred[:, 100:200]
    negative_out = y_pred[:, 200:300]
    
    pos_dist = K.sum(K.abs(anchor_out - positive_out), axis=1)
    neg_dist = K.sum(K.abs(anchor_out - negative_out), axis=1)
    
    probs = K.softmax([pos_dist, neg_dist], axis=0)
    
    return K.mean(K.abs(probs[0]) + K.abs(1.0 - probs[1]))

def contrastive_loss_patch():
        '''
    Embedding and builds the contrastive model. Prints epochs.
    
    Arguments:
         (None)
    Returns:
        siamese -- Model for contrative loss
    '''
        ds, ds_info = tfds.load('omniglot', split=['train','test'],with_info=True, as_supervised = False)
        trainDF = tfds.as_dataframe(ds[0], ds_info)
        testDF = tfds.as_dataframe(ds[1], ds_info)
        
        trainDF = trainDF[:16000]
        testDF = testDF[:4000]
        
        def make_positive_pairs(df):
            list_image = []    
            pairList= []
            
            uniqueAlph = df['alphabet'].unique()
            
            for i in range(len(uniqueAlph)):
                alphDF = df[df['alphabet'] == i]
                charUnique = alphDF['alphabet_char_id'].unique()
                for i in range(len(charUnique)):
                    dfImage = alphDF[alphDF['alphabet_char_id'] == charUnique[i]]
                    imagedf = dfImage['image'].copy()
                    imageList = imagedf.values.tolist()             
                    list_image.append(imageList[0])      
                x = 0 
                pair_list = []
                pair_target = []
                list_image2 = list_image.copy()
                for i in range(len(list_image)):
                    for x in range(len(list_image)):
                        pair_list.append([list_image[i], list_image2[x]])
                        pair_target.append(1.0)                 
                random.shuffle(pair_list)                  
            return pair_list, pair_target
        
        def get_pairs(data):
            df = data.sort_values(by=['alphabet'])
            
            df_pairs_1 = df[df.index % 2 != 0]
            
            df_pairs_2 = df[df.index % 2 != 1]
            
            df_pairs_1 = shuffle(df_pairs_1)
            
            imagedf = df_pairs_1['image'].copy()
            imageList = imagedf.values.tolist()
            
            alphabetdf = df_pairs_1['alphabet'].copy()
            alphabetList = alphabetdf.values.tolist()
            
            imagedf2 = df_pairs_2['image'].copy()
            imageList2 = imagedf.values.tolist()
            
            alphabetdf2 = df_pairs_2['alphabet'].copy()
            alphabetList2 = alphabetdf.values.tolist()
            
            imageList2 = shuffle(imageList2)
            imageList = shuffle(imageList)
            alphabetList = shuffle(alphabetList)
            alphabetList2 = shuffle(alphabetList2)
            
            trainList = []
            testList = []
            
            for i in range(len(alphabetList)):
                loopList = []
                pos = 1.0
                neg = 0.0
                if alphabetList[i] == alphabetList2[i]:
                    loopList.append(imageList[i].astype(float))
                    loopList.append(imageList2[i].astype(float))
                    testList.append(pos)
                    trainList.append(loopList)
                else:
                    loopList.append(imageList[i].astype(float))
                    loopList.append(imageList2[i].astype(float))
                    testList.append(neg)
                    trainList.append(loopList)
            
            return trainList, testList
        
        def count_neg_pairs(list_bin):
            x = 1
            count = []
            for i in range(len(list_bin)):
                if list_bin[i] == 0.0:
                    count.append(x)
                else:
                    pass
            print(len(count))
        def combine_pairs(train1, test1, train2, test2):
            for i in range(len(train2)):
                train1.append(train2[i])
            
            
            for i in range(len(test2)):
                test1.append(test2[i])
            
            shuffleList = list(zip(train1, test1))
                               
            random.shuffle(shuffleList)
            
            train1, test1 = zip(*shuffleList)
                               
            train1 = np.array(train1)
            test1 = np.array(test1)
        
            
            return train1, test1
        
        trainX, trainY = get_pairs(trainDF)
        trainXPos, trainYPos = make_positive_pairs(trainDF)
        trainYPos = trainYPos[0:20000]
        trainXFinal, trainYFinal= combine_pairs(trainXPos, trainYPos, trainX, trainY)
        testX, testY = get_pairs(testDF)
        testXPos, testYPos = make_positive_pairs(testDF)
        testXPos = testXPos[:5000]
        testYPos = testYPos[:5000]
        testXFinal, testYFinal = combine_pairs(testXPos, testYPos, testX, testY)
        lenX = round(len(trainXFinal)*0.20)
        lenY = round(len(trainYFinal)*0.20)
        X_train = trainXFinal[lenX:]
        X_test = trainYFinal[lenX:]
        y_train = trainXFinal[:lenY]
        y_test = trainYFinal[:lenY]
        X_train = X_train/255
        y_train = y_train/255
        batch_size = 96
    
        margin = 1  # Margin for constrastive loss.
        # Provided two tensors t1 and t2
        # Euclidean distance = sqrt(sum(square(t1-t2)))
        def euclidean_distance(vects):
            """Find the Euclidean distance between two vectors.
        
            Arguments:
                vects: List containing two tensors of same length.
        
            Returns:
                Tensor containing euclidean distance
                (as floating point value) between vectors.
            """
        
            x, y = vects
            sum_square = tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True)
            return tf.math.sqrt(tf.math.maximum(sum_square, tf.keras.backend.epsilon()))
        
        
        input = layers.Input((105, 105, 3))
        x = tf.keras.layers.BatchNormalization()(input)
        x = layers.Conv2D(4, (5, 5), activation="tanh")(x)
        x = layers.AveragePooling2D(pool_size=(2, 2))(x)
        x = layers.Conv2D(16, (5, 5), activation="tanh")(x)
        x = layers.AveragePooling2D(pool_size=(2, 2))(x)
        x = layers.Flatten()(x)
        
        x = tf.keras.layers.BatchNormalization()(x)
        x = layers.Dense(10, activation="tanh")(x)
        embedding_network = keras.Model(input, x)
        
        
        input_1 = layers.Input((105, 105, 3))
        input_2 = layers.Input((105, 105, 3))
        
        # As mentioned above, Siamese Network share weights between
        # tower networks (sister networks). To allow this, we will use
        # same embedding network for both tower networks.
        tower_1 = embedding_network(input_1)
        tower_2 = embedding_network(input_2)
        
        merge_layer = layers.Lambda(euclidean_distance)([tower_1, tower_2])
        normal_layer = tf.keras.layers.BatchNormalization()(merge_layer)
        output_layer = layers.Dense(1, activation="sigmoid")(normal_layer)
        siamese = keras.Model(inputs=[input_1, input_2], outputs=output_layer)
        
        def loss(margin=1):
            """Provides 'constrastive_loss' an enclosing scope with variable 'margin'.
        
          Arguments:
              margin: Integer, defines the baseline for distance for which pairs
                      should be classified as dissimilar. - (default is 1).
        
          Returns:
              'constrastive_loss' function with data ('margin') attached.
          """
        
            # Contrastive loss = mean( (1-true_value) * square(prediction) +
            #                         true_value * square( max(margin-prediction, 0) ))
            def contrastive_loss(y_true, y_pred):
                """Calculates the constrastive loss.
        
              Arguments:
                  y_true: List of labels, each label is of type float32.
                  y_pred: List of predictions of same length as of y_true,
                          each label is of type float32.
        
              Returns:
                  A tensor containing constrastive loss as floating point value.
              """
        
                square_pred = tf.math.square(y_pred)
                margin_square = tf.math.square(tf.math.maximum(margin - (y_pred), 0))
                return tf.math.reduce_mean(
                    (1 - y_true) * square_pred + (y_true) * margin_square
                )
        
            return contrastive_loss
        
        siamese.compile(loss=loss(margin=margin), optimizer="RMSprop", metrics=["accuracy"])
        history = siamese.fit(
                [X_train[:,0], X_train[:,1]],
                X_test[:],
                validation_data=([y_train[:,0], y_train[:,1]], y_test[:]),
                batch_size=96,
                epochs=epochs,
            callbacks=[callback])
        
        def get_result(X, Y):
            predictions = []
            counts = 0
            for i in range(len(X)):
                prediction = siamese.predict([X[i][0].reshape(-1, 105, 105, 3), X[i][1].reshape(-1, 105, 105, 3)])

                if prediction > 0.5:
                    predictions.append(1)
                else:
                    predictions.append(0)
            for i in range(len(X)):
                if predictions[i] == Y[i]:
                    counts = counts+1
            accurracy = (counts/len(X))*100
                    
            return accurracy 
        
        test_acc = get_result(testXFinal, testYFinal)
        train_acc = get_result(trainXFinal, trainYFinal)
        #mixX = np.concatenate((trainXFinal[:1000],testXFinal[:1000]))
        #mixY = np.concatenate((testYFinal[:1000],trainYFinal[:1000]))
        
        #mix_acc = get_result(mixX, mixY)
        print("train accuracy is {}%".format(train_acc))
        print("test accuracy is {}%".format(test_acc))
        #print("mix of train and test accuracy is {}%".format(mix_acc))
    
        return siamese
        
    

def triplet_embedding(tripletLitst, label,testList, x_test, y_test, x_train, y_train):
    '''
    Embedding and builds the triplet model. Prints epochs.
    
    Arguments:
        tripletLitst, label,testList, x_test, y_test, x_train, y_train
    Returns:
        triplet_model -- Model for triplet loss
    '''
    tripletLitst = np.array(tripletLitst)
    tripletLitst = tripletLitst.reshape(len(tripletLitst), 3, 105, 105, 3)
    testList = testList.reshape(len(testList), 3, 105, 105, 3)
    
    x_test = np.array(x_test)
    y_test = np.array(y_test)
    label = np.array(label)
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    
    input_layer = Input((105, 105, 3))
    x = Conv2D(32, 3, activation='relu')(input_layer)
    x = Conv2D(32, 3, activation='relu')(x)
    x = MaxPool2D(2)(x)
    x = Conv2D(64, 3, activation='relu')(x)
    x = Conv2D(64, 3, activation='relu')(x)
    x = MaxPool2D(2)(x)
    x = Conv2D(128, 3, activation='relu')(x)
    x = Flatten()(x)
    x = Dense(100, activation='relu')(x)
    model = Model(input_layer, x)
    
    triplet_model_a = Input((105, 105, 3))
    triplet_model_p = Input((105, 105, 3))
    triplet_model_n = Input((105, 105, 3))
    triplet_model_out = Concatenate()([model(triplet_model_a), model(triplet_model_p), model(triplet_model_n)])
    triplet_model = Model([triplet_model_a, triplet_model_p, triplet_model_n], triplet_model_out)
    
    triplet_model.compile(loss=triplet_loss, optimizer='adam')
    history = triplet_model.fit([x_train[:,0],
                   x_train[:,1], x_train[:,2]], 
                  x_test[:], validation_data=([y_train[:,0],
                   y_train[:,1], y_train[:,2]], 
                  y_test[:]), batch_size = 96, epochs=epochs, 
                  callbacks=[callback])
    #history.history['val_loss']
    
    return triplet_model


def triplet_acc(imageTrips):
    '''
    Run the model and let model conduct prediction 
    
    Argument:
        imageTrips: input image
        
    Return:
        result: prediction results
    '''
    imageTrips = imageTrips.reshape(len(imageTrips), 3, 105, 105, 3)
    result = []
    predictionP = triplet_model.layers[3].predict(imageTrips[:,1])
    predictionN = triplet_model.layers[3].predict(imageTrips[:,2])
    
    for i in range(len(imageTrips)):
        posDist = sum(predictionP[i])
        negDist = sum(predictionN[i])
        if posDist > negDist:
            result.append(0)
        if posDist < negDist:
            result.append(1)
    return result

def Results(tripletLitst, testList, testlabel, label):
    '''
    Arguments: 
        tripletLitsts -- triplet data list
        testList -- test data list
        testlabel -- test label list
        label -- training label list
       
    Return:
        results -- accuaracy when performed on tripletLitst
        resultsTest -- accuaracy when performed on testList
        resultsMix -- accuracy when performed on the mixt of trpletLitst abd testList
        mixX -- a combined NumPy array of testList (from the 1st to the 1000th elements) and triplet litst 
        (from the 1st to the 1000th elements).        
        mixY -- a combined NumPy array of testLable (from the 1st to the 1000th elements) and label
        (from the 1st to the 1000th elements).
    '''
    results = triplet_acc(tripletLitst)
    resultsTest = triplet_acc(testList)
    
    mixX = np.concatenate((testList[:1000], tripletLitst[:1000]))
    mixY = np.concatenate((testlabel[:1000], label[:1000]))
    
    resultsMix = triplet_acc(mixX)
    
    return results, resultsTest, resultsMix, mixX, mixY


def get_acc(predY, trueY):
    '''
    Calculate accuracy of CNN with triplet loss
    
    Arguments:
        predY -- predicted value
        trueY -- the true value
        
    Return:
        accuracy -- the accuracy of the model with triplet loss function
    '''
    count = 0
    for i in range(len(predY)):
        if predY[i] == trueY[i]:
            count = count+1
        else:
            pass
    accuracy = (count/len(predY))*100
    return accuracy 


def print_acc(results, label, resultsTest, testlabel, resultsMix, mixY):
    '''
    Print out the accuracy of triplet loss CNN for each types of results
    
    Arguments:
        results -- accuaracy when performed on tripletLitst
        label -- training label list.
        resultsTest -- accuaracy when performed on testList
        testlabel -- test label list
        resultsMix -- accuracy when performed on the mixt of trpletLitst abd testList
        mixY -- a combined NumPy array of testLable (from the 1st to the 1000th elements) and label
        (from the 1st to the 1000th elements).
        
    Print:
        "Accuracy for triplet loss CNN on {dataTypes} is {dataPrediction}%". It prints out the accuracy 
        for triplet loss CNN on each data type
        
    '''
    train_prediction_acc = get_acc(results, label)
    print("Accuracy for triplet loss CNN on training data is {}%".format(train_prediction_acc))
    test_prediction_acc = get_acc(resultsTest, testlabel)
    print("Accuracy for triplet loss CNN on test data is {}%".format(test_prediction_acc))
    #mix_prediction_acc = get_acc(resultsMix, mixY)
    #print("Accuracy for triplet loss CNN on mixed data of trainig and test data is {}%".format(mix_prediction_acc))


In [3]:
if __name__ == '__main__':
    print(my_team())
    #Below should print Triplet loss computed predictions
    triplet_loss_test()
    
    #Below should assign data_prepare()
    arrayImage, arrayAlph, x_test, y_test, x_train, y_train = data_prepare()
    
    #Below should assign global_variable1 &2
    label, tripletLitst = global_variable1(arrayImage, arrayAlph)
    testlabel, testList = global_variable2(arrayImage, arrayAlph)
    
    #Below should assign dataProcess()
    arrayImage, arrayAlph, x_test, y_test, x_train, y_train = dataProcess(arrayImage, arrayAlph, 
                                                                          x_test, y_test, 
                                                                          tripletLitst)
    
    print("Triplet Loss epochs processing. Now building model...")
    triplet_model = triplet_embedding(tripletLitst, label,testList, x_test, y_test, x_train, y_train)
    print("Contrative Loss epochs processing. Now building model...")
    siamese_model = contrastive_loss_patch()
    
    #Below should assign Results()"
    results, resultsTest, resultsMix, mixX, mixY = Results(tripletLitst, testList, testlabel, label)    
    #Below should print out 3 Accuracy for triplet loss CNN
    print_acc(results, label, resultsTest, testlabel, resultsMix, mixY)

[(9993304, 'Brendan', 'Wallace-Nash'), (9300449, 'Min-Pu', 'Tsai'), (10661450, 'Bingqing', 'Qian')]
Triplet loss computed with numpy 0.2633540523052216
Metal device set to: Apple M1 Pro
Triplet loss computed with tensorflow 0.26335403


2022-05-11 10:06:48.290988: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-11 10:06:48.291081: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-05-11 10:06:48.292689: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-05-11 10:06:48.292735: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-05-11 10:06:48.303460: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been bu

Triplet Loss epochs processing. Now building model...
Train on 16000 samples, validate on 4000 samples
Epoch 1/10


2022-05-11 10:07:04.477342: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-11 10:07:04.477365: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-05-11 10:07:04.488475: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-05-11 10:07:04.508721: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-05-11 10:07:04.603680: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


16000/16000 [==============================] - ETA: 0s - loss: 0.2864

...

22400/22400 [==============================] - 20s 887us/sample - loss: 0.2923 - accuracy: 0.5274 - val_loss: 0.2867 - val_accuracy: 0.5098
Epoch 2/10
22400/22400 [==============================] - 14s 640us/sample - loss: 0.2274 - accuracy: 0.6505 - val_loss: 0.2126 - val_accuracy: 0.7130
Epoch 3/10
22400/22400 [==============================] - 14s 621us/sample - loss: 0.2059 - accuracy: 0.7215 - val_loss: 0.2015 - val_accuracy: 0.7296
Epoch 4/10
22400/22400 [==============================] - 14s 636us/sample - loss: 0.2005 - accuracy: 0.7245 - val_loss: 0.1987 - val_accuracy: 0.7296
Epoch 5/10
22400/22400 [==============================] - 14s 641us/sample - loss: 0.1994 - accuracy: 0.7245 - val_loss: 0.1984 - val_accuracy: 0.7296
Epoch 6/10
22400/22400 [==============================] - 15s 667us/sample - loss: 0.1990 - accuracy: 0.7245 - val_loss: 0.1982 - val_accuracy: 0.7296
Epoch 7/10
22400/22400 [==============================] - 14s 642us/sample - loss: 0.1991 - accuracy: 0.7

/Users/brendanwallace-nash/tensorflow-test/env/lib/python3.9/site-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2022-05-11 10:16:48.228820: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


train accuracy is 72.55%
test accuracy is 73.14285714285714%
mix of train and test accuracy is 74.95%


2022-05-11 10:19:41.558190: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Accuracy for triplet loss CNN on training data is 82.78999999999999%
Accuracy for triplet loss CNN on test data is 67.675%
Accuracy for triplet loss CNN on mixed data of trainig and test data is 74.2%
